<a href="https://colab.research.google.com/github/RayBasilio123/ic2020/blob/main/WindSun_SARIMAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo SARIMA para Previsão de Séries Temporais com base nos dados do INMET

## Obtendo os dados atráves do github

In [43]:
#imports
import pandas as pd
import numpy as np
from datetime import timedelta
import statsmodels.api as sm

In [45]:
#obtem os dados
url1 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_ventovel_pampulha_2018_2019'
url2 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_radiacao_pampulha_2018_2019'
url3 = 'https://raw.githubusercontent.com/RayBasilio123/R5/master/Eto'

df_ventovel = pd.read_csv(url1, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_radiacao = pd.read_csv(url2, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_eto = pd.read_csv(url3, header=0, parse_dates=[0], index_col=0, squeeze=True)

series_ventovel = pd.Series(df_ventovel)
series_radiacao = pd.Series(df_radiacao)
series_eto = pd.Series(df_eto)



In [48]:
series_ventovel=series_ventovel.resample('D').mean()
series_radiacao=series_radiacao.resample('D').mean()
series_eto = pd.Series(df_eto)


## SARIMAX

In [49]:
def run_sarima(series,exog,steps_ahead,config_ordem,config_sazonal):
  result = []
  
  #Lista de data+hora que será previsto
  begin = series.index.max() + timedelta(days=0)
  date_list = [begin + timedelta(hours=x) for x in range(1,steps_ahead+1)]
  
  #Valores da série
  values = series.values
  
  #Valores da variável exogena
  ex = exog.values

  #Valores da variável exogena que será prevista
  ex_cast = ex.reshape(-1, 1)[-steps_ahead:]
  
  #ARIMA
  mod = sm.tsa.statespace.SARIMAX(values, exog=ex, order=config_ordem, seasonal_order=config_sazonal)
  res = mod.fit(disp=False)
  forecast = res.forecast(steps=steps_ahead, exog=ex_cast)

  #Resultado no formato para ser exibido no gráfico
  for i in range(steps_ahead):
    if forecast[i] < 0: 
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),0])
    else:
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),round((forecast[i]),3)])

  return result
  

In [50]:
def run_sarimaDay(series,exog,steps_ahead,config_ordem,config_sazonal):
  result = []
  
  #Lista de data+hora que será previsto
  begin = series.index.max() + timedelta(days=0)
  date_list = [begin + timedelta(days=x) for x in range(1,steps_ahead+1)]
  
  #Valores da série
  values = series.values
  
  #Valores da variável exogena
  ex = exog.values

  #Valores da variável exogena que será prevista
  ex_cast = ex.reshape(-1, 1)[-steps_ahead:]
  
  #ARIMA
  mod = sm.tsa.statespace.SARIMAX(values, exog=ex, order=config_ordem, seasonal_order=config_sazonal)
  res = mod.fit(disp=False)
  forecast = res.forecast(steps=steps_ahead, exog=ex_cast)

  #Resultado no formato para ser exibido no gráfico
  for i in range(steps_ahead):
    if forecast[i] < 0: 
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),0])
    else:
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),round((forecast[i]),3)])

  return result
  

In [51]:
order_ven = (3,1,0) 
order_ven_sazonal = (6,0,0,4)
order_rad = (4,0,2)
order_rad_sazonal = (1,0,2,4)
order_eto2 = (5, 1, 2) #graficos
order_eto2_sazonal = (1,0,1,3)


## Previsão 1 hora a frente

In [ ]:
result_ven_1 = run_sarima(series_ventovel,series_radiacao,1,order_ven, order_ven_sazonal)
result_ven_1

In [ ]:

result_rad_1 = run_sarima(series_radiacao,series_ventovel,1,order_rad,order_rad_sazonal)
result_rad_1

## Previsão 1 dia a frente

In [ ]:
result_ven_1 = run_sarima(series_ventovel,series_radiacao,24,order_ven, order_ven_sazonal)
result_ven_1

In [ ]:
result_rad_1 = run_sarima(series_radiacao,series_ventovel,24,order_rad,order_rad_sazonal)
result_rad_1

In [53]:
result_Eto_1 = run_sarimaDay(series_eto,series_radiacao,1,order_eto2,order_eto2_sazonal)
result_Eto_1

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[['01/01/2020 00:00:00', 5.694]]

## Previsão 3 dias a frente 

In [ ]:
result_Eto_1 = run_sarimaDay(series_eto,series_radiacao,3,order_eto2,order_eto2_sazonal)
result_Eto_1

## Previsão 7 dias a frente 

In [ ]:
result_ven_1 = run_sarima(series_ventovel,series_radiacao,168,order_ven, order_ven_sazonal)
result_ven_1

In [ ]:
result_rad_1 = run_sarima(series_radiacao,series_ventovel,168,order_rad,order_rad_sazonal)
result_rad_1

In [54]:
result_Eto_1 = run_sarimaDay(series_eto,series_radiacao,7,order_eto2,order_eto2_sazonal)
result_Eto_1

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[['01/01/2020 00:00:00', 5.933],
 ['02/01/2020 00:00:00', 5.542],
 ['03/01/2020 00:00:00', 5.603],
 ['04/01/2020 00:00:00', 5.358],
 ['05/01/2020 00:00:00', 5.288],
 ['06/01/2020 00:00:00', 5.563],
 ['07/01/2020 00:00:00', 5.731]]

## Previsão 10 dias a frente 

In [55]:
result_Eto_1 = run_sarimaDay(series_eto,series_radiacao,10,order_eto2,order_eto2_sazonal)
result_Eto_1

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[['01/01/2020 00:00:00', 5.822],
 ['02/01/2020 00:00:00', 4.904],
 ['03/01/2020 00:00:00', 4.603],
 ['04/01/2020 00:00:00', 5.964],
 ['05/01/2020 00:00:00', 5.551],
 ['06/01/2020 00:00:00', 5.585],
 ['07/01/2020 00:00:00', 5.364],
 ['08/01/2020 00:00:00', 5.285],
 ['09/01/2020 00:00:00', 5.566],
 ['10/01/2020 00:00:00', 5.73]]